In [ ]:
import ollama
from IPython.display import Markdown, display
import requests
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_core.messages import AIMessage, HumanMessage
from typing_extensions import TypedDict
from typing import Literal
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from IPython.display import Image, display
from pprint import pprint

In [ ]:
llm = ChatOllama(
    model="llama3.2:1b",
    temperature=0,
)

In [ ]:
class State(TypedDict, total=False):
    code: str
    summary: str
    result: str
    authors: str
    categories: str
    source_repo: str
    contact: str

In [ ]:
def get_wrapper_script(state):
    with open('input/sample_wrapper.R') as f:
        script = f.read()
        state['code'] = script
    return state

def summarize(state):
    if "summary" in state and state['summary']:
        feedback = input('Give feedback. If good, leave blank.')
        response = llm.invoke(f"""
Improve the following summary based on the feedback:

Summary: {state['summary']}

Feedback: {feedback}""")
    else:
        response = llm.invoke(f"""
This code is being used to build a drag-and-drop user experience for bioinformatics analysis.

Please create a no-code description, including details such as parameters,
parameter details, default values, parameter types, input file types, output file types, etc.                       
Code: {state['code']}
""")
    state['summary'] = response
    print('--------')
    print(state['summary'].content)
    print('---------')
    return state

def clear_summary_feedback(state):
    state['feedback'] = ''
    return state

def get_authors(state):
    authors = input('Who are the authors of this module?')
    state['authors'] = authors
    return state

def get_categories(state):
    categories = input('What are the categories of this module?')
    state['categories'] = categories
    return state

def get_source_repo(state):
    source_repo_link = input('Add a link to the source repo')
    source_repo_link_title = llm.invoke(
        f'Create a title for this link: {source_repo_link}'
    ).content
    state['source_repo'] = f'''Link: {source_repo_link}
Title: {source_repo_link_title}'''
    return state

def get_contact(state):
    contact_link = input('Add a link to a contact page')
    contact_link_title = llm.invoke(
        f'Create a title for this link: {contact_link}'
    ).content
    state['contact'] = f'''Link: {contact_link}
Title: {contact_link_title}'''
    return state


def final_readme(state):
    if "result" in state and state['result']:
        feedback = input('Give feedback. If good, leave blank.')
        response = llm.invoke(f"""Improve the following readme based on the feedback:

Current readme: {state['result']}

Feedback: {feedback}""")
    else:
        response = llm.invoke(f"""
Generate a Markdown file with this information:
                              
Description
Module Details
-Authors
-Categories
-Source Repo
-Contact
Input Files (info + type)
Output Files (info + type)
Parameters (formatted as a Markdown table with name, description, default value, and type)

Here is the content you can use:  
Summary: {state['summary']}
Authors: {state['authors']}
Categories: {state['categories']}
Source Repo: {state['source_repo']}
Contact: {state['contact']}
""")
    state['result'] = response
    print('--------')
    print(state['result'].content)
    print('---------')
    return state

def save_readme(state):
    with open('output/README.md', 'w') as file:
        file.write(state['result'].content)

    print("Markdown file 'README.md' created successfully.")
    return state

In [ ]:
def judge_summary(state) -> Literal["clear_summary_feedback", "summarize"]:
    approved = input("Enter 'yes' if you approve of the summary")
    if approved == 'yes':
        return "clear_summary_feedback"
    return "summarize"

def judge_readme(state) -> Literal["save_readme", "final_readme"]:
    approved = input("Enter 'yes' if you approve of the README")
    if approved == 'yes':
        return "save_readme"
    return "final_readme"

In [ ]:
builder = StateGraph(State)
builder.add_node("get_wrapper_script", get_wrapper_script)
builder.add_node("summarize", summarize)
builder.add_node("clear_summary_feedback", clear_summary_feedback)
builder.add_node("get_authors", get_authors)
builder.add_node("get_categories", get_categories)
builder.add_node("get_source_repo", get_source_repo)
builder.add_node("get_contact", get_contact)
builder.add_node("final_readme", final_readme)
builder.add_node("save_readme", save_readme)

builder.add_edge(START, "get_wrapper_script")
builder.add_edge("get_wrapper_script", "summarize")
builder.add_conditional_edges("summarize", judge_summary)
builder.add_edge("clear_summary_feedback", "get_authors")
builder.add_edge("get_authors", "get_categories")
builder.add_edge("get_categories", "get_source_repo")
builder.add_edge("get_source_repo", "get_contact")
builder.add_edge("get_contact", "final_readme")
builder.add_conditional_edges("final_readme", judge_readme)
builder.add_edge("save_readme", END)

# Add
graph = builder.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
response = graph.invoke({"summary": ""})

In [ ]:
with open('output/README.md', 'r') as file:
    display(Markdown(file.read()))